# Translating conditional logit function to Python
### Author: Lauri Kytömaa
### Date created: January, 22, 2019

### This Jupyter notebook translates Aguirregabiria and Mira's Gauss code named "clogit.src" into Python. This code estimates McFadden's Conditional Logit model using Newton's method with analytical gradient and the hessian.

The original Gauss file can be found in the original author's Nested Pseudo Likelihood (NPL) zip file/n* **AM code link**:
http://individual.utoronto.ca/vaguirre/wpapers/program_code_survey_joe_2008.html
* **McFadden theory:** https://eml.berkeley.edu/reprints/mcfadden/zarembka.pdf, https://epubs.siam.org/doi/pdf/10.1137/0908006

-------------

## Parametric set-up

Individual $i$'s latent utility for alternative $j \in \{1,\dots,J\}$ is given by: 
\begin{equation*}
y_{ij} = z_{ij1}\beta_1 +z_{ij2}\beta_2 + z_{ij3}\beta_3 + r_{ij} + \varepsilon_{ij}
\end{equation*}
Where:

* $y_{ij}$: Latent utility for individual $i$ and alternative $j$.
* $\beta_k$: Parameter $k$. Associated with variable $z_{ijk}$.
* $z_{ijk}$: Variable associated with $\beta_k$ for individual $i$ and alternative $j$.
* $r_{ij}$: Individual fixed effect for individual $i$ and alternative $j$.
* $\varepsilon_{ij}$: Random extreme value shock for individual $i$ and alternative $j$.

$y_{ij}$ will not be observable in practice, instead we will observe binary outcomes $Y_{ij} \in \{0,1\}$ that reflect a consumer's choice across alternatives. We will have $Y_{ij} = 1$ when $y_{i,j} \geq y_{i,-j} \forall -j \neq j$.

## Simulated data to test conditional logit function
* Number of alternatives, $J = 5$
* Number of observations, $N = 2,000$
* Number of parameters, $K = 3$
* True parameters values: $[\beta_1, \beta_2 ,\beta_3] = [1, 2, 3] $
* $z_{ijk} \sim N(1, \sigma_z^2)$ where $\sigma_z$ is drawn from $2*U[0,1]$
* $r_{ij} \sim N(0, \sigma_\varepsilon^2)$ were $\sigma_r$ is drawn from $U[0,1]$
* $\varepsilon_{ij} \sim$ Extreme Value Type I. Draws taken by taken random draws from the standard uniform and then transformed: $-log(-log(U[0,1]))$. See: Train 2009, Discrete Choice Methods with Simulation

In [1]:
# Import libraries 
import numpy as np

# ======= Constants =======

# Number of alternatives
nalt = 5

# Number of observations
nobs = 2_000

# Number of parameters
npar = 3

meanz = np.ones((nalt*npar, 1)) # Mean set to 1. 
sdz = 2 * np.random.uniform(size=(nalt*npar, 1)) # Set a different std for each alt/parameters combo
meanr = np.zeros((nalt, 1)) # Mean zero. 
sdr = np.random.uniform(size=(nalt, 1)) # Set a different standard deviation for each alt

# True parameters
true_params = np.array([[1.], [2.], [3.]])

# Simulations
zobs = meanz.T + sdz.T*np.random.normal(size=(nobs, nalt*npar))
robs = meanr.T + sdr.T*np.random.normal(size=(nobs, nalt))
eps = -np.log(-np.log(np.random.uniform(size=(nobs, nalt))))
yobs = np.zeros((nobs, nalt))

j = 0
while j < nalt:
    yobs[:, [j]] = zobs[:, npar*j:npar*(j+1)]@true_params + robs[:, [j]] + eps[:, [j]]

    j += 1
    
yobs = np.argmax(yobs, axis=1)
yobs = np.reshape(yobs, (len(yobs), 1))

test_elements = np.arange(0, nalt, 1)
print("Testing if yobs fall in correct range (should be 1): " + str((np.isin(yobs, test_elements)*1).min()))

# Name my parameters 
namesb = np.array([["b1"], ["b2"], ["b3"]])

print(yobs.shape, zobs.shape, robs.shape, namesb.shape)

Testing if yobs fall in correct range (should be 1): 1
(2000, 1) (2000, 15) (2000, 5) (3, 1)


## Conditional logit function

In [2]:
def clogit(ydum, x, restx, par_names):
    """
    Maximum likelihood estimation of a multinomial logit
    _________________________________
    
    ----Inputs----
        
    ydum: (N x 1) vector of observations of dependent variable, these will
    have values {0, ..., nalt}. I start from 0, unlike AM code. 
    This is because python lists are 0-indexed.

    x: (N x (k * nalt)) matrix of explanatory variables associated with 
    unrestricted parameters. Firt k columns correspond to alternative 0. 
    
    restx: (N x nalt) vector of the sum of the explanatory variables whose
    parameters are restricted to be equal to 1. 
    
    par_names: (k x 1) vector with names of our parameters
    
    ----Ouputs----
    
    b0: (k x 1) vector with ML estimates

    Avarb: (k x k) matrix with estimate of the covariance matrix 
    
    """    
    cconvb = 1e-6
    myzero = 1e-16
    nobs = ydum.shape[0]
    nalt = int(np.max(ydum) + 1)
    npar = int(x.shape[1]/nalt)
    
    # Error check 
    if npar != par_names.shape[0]:
        return "Error: The dimensions of x and names of parameters do not match"

    xysum = 0
    
    j = 0
    while j < nalt:
        xysum = xysum + (((ydum==j)*1)*x[:, npar*j:npar*(j + 1)]).sum(axis = 0)
        j += 1
    
    xysum = np.expand_dims(xysum, axis = 1)
    
    iter = 2  # Number of iterations
    criter = 1000
    llike = -nobs
    b0 = np.ones((npar, 1))
    
    while criter > cconvb:
        
        #-----Computing probabilities-----#
        phat = np.zeros((nobs, nalt))
        
        j = 0
        while j < nalt:
            phat[:, [j]] = x[:, npar*j:npar*(j + 1)]@b0 + restx[:, [j]]
            j += 1
        
        phat_maxs = np.max(phat.T, axis=0)
        phat_maxs = np.expand_dims(phat_maxs, axis=1) 

        phat = phat - phat_maxs
        
        sumexp = np.exp(phat.T).sum(axis=0)
        sumexp = np.expand_dims(sumexp, axis=1)
        
        phat = np.exp(phat)/sumexp
        
        #-----Computing xmean-----#
        sumpx = np.zeros((nobs, 1))
        xxm = 0
        llike = 0 # llike is the Log-likelihood function
        
        j = 0
        while j < nalt:
            xbuff = x[:, npar*j: npar*(j + 1)]
            sumpx = sumpx + phat[:, [j]]*xbuff
            xxm = xxm + (phat[:, [j]]*xbuff).T @ xbuff
            llike = (llike + (1*(ydum ==j)*np.log(((phat[:,[j]] > myzero)*1)*phat[:,[j]]
                                     + ((phat[:, [j]] < myzero)*1)*myzero).sum(axis=0)))
            j += 1

        #-----Computing gradient-----#
        d1llike = xysum - sumpx.sum(axis=0, keepdims=True).T
        
        #-----Computing Hessian-----#
        d2llike = -(xxm - sumpx.T @ sumpx) # 
        
        #-----Gauss iteration-----#
        
        b1 = b0 - np.linalg.inv(d2llike) @ d1llike
        criter = np.sqrt((b1 - b0).T @ (b1 - b0)) 
        b0 = b1
        iter = iter + 1
        
        Avarb = np.linalg.inv(-d2llike)
        
        sdb = np.sqrt(np.diag(Avarb))
        sdb = np.expand_dims(sdb, axis = 1)
        
        tstat = b0/sdb
        
        y_alts = np.arange(0, nalt, 1)
        y_alts = np.expand_dims(y_alts, axis=1)
        
        numyj = ((ydum == y_alts.T)*1).sum(axis=0) # Shape of ydum is important to review
        logL0 = (numyj*np.log(numyj/nobs)).sum(axis=0)
        logL0 = np.expand_dims(logL0, axis = 0)
        
        lrindex = 1 - llike/logL0 # Likelihood ratio index
        
        #-----Prints-----#
        print("Iteration: "+str(iter))
        print("--------------------------------------------------")
        print("Parameter  estimate   Standard Errors   t-ratios")
        print("--------------------------------------------------")
        
        j = 0
        while j < npar:
            print(par_names[j], b0[j], sdb[j], tstat[j])
            j += 1
        print("     ")
        
    return b0, Avarb

# Estimation

Estimation of the parameters using the simulated data is presented below. One can see that the estimates approach the true parameters as we take further iterations.

In [3]:
b0, Avarb = clogit(yobs, zobs, robs, namesb)

Iteration: 3
--------------------------------------------------
Parameter  estimate   Standard Errors   t-ratios
--------------------------------------------------
['b1'] [0.71851404] [0.03216837] [22.33604128]
['b2'] [1.34519691] [0.04190127] [32.10396235]
['b3'] [1.86787711] [0.03724035] [50.15735229]
     
Iteration: 4
--------------------------------------------------
Parameter  estimate   Standard Errors   t-ratios
--------------------------------------------------
['b1'] [0.8916201] [0.03137732] [28.41606724]
['b2'] [1.81496373] [0.05042722] [35.99174585]
['b3'] [2.62094486] [0.05613745] [46.68798926]
     
Iteration: 5
--------------------------------------------------
Parameter  estimate   Standard Errors   t-ratios
--------------------------------------------------
['b1'] [1.02321413] [0.03818854] [26.79374716]
['b2'] [2.12676722] [0.06571406] [32.36396214]
['b3'] [3.09949095] [0.08031679] [38.59082289]
     
Iteration: 6
--------------------------------------------------
Para